In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_sub = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv')
df_train = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
df_test = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')

In [ ]:
df_train.head(3)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import string
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Flatten

In [ ]:
input_text= (df_train.anchor + ' ' + df_train.context + ' ' + df_train.target).str.lower()

In [ ]:
token = Tokenizer()

In [ ]:
token.fit_on_texts(input_text)

In [ ]:
vocab_size = len(token.word_index) + 1

In [ ]:
embedding_doc = token.texts_to_sequences(input_text)

In [ ]:
max([len(i) for i in embedding_doc])

In [ ]:
max_length = 20
padded_docs = pad_sequences(embedding_doc, maxlen=max_length, padding='post')

In [ ]:
y_train = pd.get_dummies(df_train.score).values

In [ ]:
e = tf.keras.layers.Embedding(vocab_size, 50, input_length = max_length)

In [ ]:
# model = Sequential()
# model.add(e)
# model.add(Conv1D(32,3, activation='relu'))
# model.add(Conv1D(64,3, activation='relu'))
# model.add(Flatten())
# model.add(Dense(y_train.shape[1], activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# print(model.summary())

In [ ]:
# model.fit(padded_docs, y_train, epochs=5, verbose=1, validation_split=0.3)

In [ ]:
# res = model.predict(pad_sequences(token.texts_to_sequences(df_test.id + ' '+ df_test.anchor + ' ' + df_test.context + ' ' + df_test.target
#                                                           ), maxlen=max_length, padding='post'))

In [ ]:
# res = pd.Series(res.argmax(1)).replace(dict(zip(range(0,5),pd.get_dummies(df_train.score).columns.tolist())))

In [ ]:
# res_train = pd.Series(model.predict(padded_docs).argmax(1)).replace(dict(zip(range(0,5),pd.get_dummies(df_train.score).columns.tolist())))
# np.corrcoef(res_train, df_train.score.tolist())

In [ ]:
# df_test['score']=res
# df_test[['id', 'score']].to_csv('submission.csv', index=False)

In [ ]:
################## Trying Spacy ###############
import spacy

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
similar_score = []
for i in range(0,len(df_train)):
    token1 = nlp(df_train.anchor.iloc[i])
    token2 = nlp(df_train.target.iloc[i])
    s = token1.similarity(token2)
    if i%5000==0:
        print(token1.text, token2.text, s)
    similar_score.append(s)

In [ ]:
df_train['similarity_score'] = similar_score

In [ ]:
similar_test_score = []
for i in range(0,len(df_test)):
    token1 = nlp(df_test.anchor.iloc[i])
    token2 = nlp(df_test.target.iloc[i])
    s = token1.similarity(token2)
    if i%5000==0:
        print(token1.text, token2.text, s)
    similar_test_score.append(s)

In [ ]:
df_test['score']=similar_test_score
df_test[['id', 'score']].to_csv('submission.csv', index=False)